In [111]:
import pandas as pd
import os
#import matplotlib.pyplot as plt
from scipy import signal
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import resample
import random
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

def get_frequency(data):    
    if(data.columns._data[0]=='Time (s)'):        
        time = data['Time (s)']
        num_entries = len(time)    
        freq = (num_entries-1)/time[num_entries-2]
        return freq
    
    elif(data.columns._data[0]=='Time(s)'):
        time = data['Time(s)']
    
        num_entries = len(time)    
        freq = (num_entries-1)/time[num_entries-2]
        return freq
    
    else: 
        print("Data error")

# Butterwort filter
def butterfilt_data(sensordata, fs=200, fc=5):
    sensordata_filt = np.zeros(sensordata.shape)
    w = 0.05
    b, a = signal.butter(fc,w, 'low')
    for i in range (3):
        sensordata_filt[:,i] = signal.filtfilt(b,a, sensordata[:,i])  
    return sensordata_filt

#sg filter
def sgfilt_data(sensordata, win_len = 51):
    sensordata_filt = np.zeros(sensordata.shape)
    if not(int(win_len)%2):
        win_len -= 1
    for i in range (0,3):
        sensordata_filt[:,i] = signal.savgol_filter(sensordata[:,i], int(win_len), 2)
    return sensordata_filt

def cut_data(fil_data, freq):
    gyr_abs = np.linalg.norm(fil_data,axis=1)
    #determining the correct height still needs some playing/automation
    #idea: get values of all peaks, take mean as value
    #max_v = np.max(fil_data[:,0])
    srt = np.sort(fil_data[:,0])    
    max_v2 = srt[int(np.floor(0.95*len(srt)))]
    peaks, _ = signal.find_peaks(gyr_abs,height=max_v2/2, distance=freq/2)
    if len(peaks) <= 1: return [],[]
    diff_peaks = np.diff(peaks)
    n = int(np.ceil(len(diff_peaks)/5))
    gap1  = np.argmax(diff_peaks[:n])
    gap2  = np.argmax(diff_peaks[-n:])    
    gap2  = int(gap2 + np.shape(diff_peaks)-n)    
    gyr_cut = fil_data[peaks[gap1+1]:peaks[gap2],:]
    gyr_cut_fil = fil_data
    gyr_cut_fil[0:peaks[gap1],:] = 0    
    gyr_cut_fil[peaks[gap2]:len(fil_data),:] = 0
    return gyr_cut, gyr_cut_fil

def rotate_data(gyr_cut):
    
    pca = PCA(n_components=3)
    rot_data = pca.fit_transform(gyr_cut)
    a = pca.explained_variance_ratio_
    b= pca.components_        
    rot_data = MinMaxScaler().fit_transform(rot_data)       
    return rot_data

#just for testing the inbuilt rotation function, w returns the eigenvectors/principal components
def pca_man(gyr_cut):    
    x1 = np.mean(gyr_cut,axis=0)   
    n = len(gyr_cut)
    for i in range (0,n):
        gyr_cut[i,:] -= x1    
    X = np.transpose(gyr_cut)
    J = np.identity(n)-1/n * np.ones(n)
    C = np.matmul(np.matmul(X,J),np.transpose(np.matmul(X,J)))
    w,v = np.linalg.eig(C)
    
def sample_data(data,freq) :
    srt = np.sort(data[:,0])    
    max_v2 = srt[int(np.floor(0.95*len(srt)))]
    peaks, _ = signal.find_peaks(data[:,0],height = max_v2 * 0.9, distance=freq/2)
    samples = []
    for i in range (0,len(peaks)-1):        
        sample = data[peaks[i]:peaks[i+1]+1,:]
        samples.append(sample)
    return samples

def resample_data(samples, n):
    num_samples = len(samples)       
    for i in range (0,num_samples):
        samples[i] = resample(samples[i],n)
    return samples

def remove_bad_samples(samples,n, min_samples):
    num_samples = len(samples)
    data_per_time = np.empty((num_samples,n))
    for i in range (0,num_samples):
       for j in range (0,n):
        data_per_time[i,j]=samples[i][j,0]
    means = np.median(data_per_time,axis=0)
    not_in_av = np.zeros((num_samples,n),dtype = bool)
    for i in range (0,num_samples):
       for j in range (0,n):
           if(abs(data_per_time[i,j]-means[j]) > 0.2 * means[j]):
               not_in_av[i,j] = 1
    num_wrong = np.count_nonzero(not_in_av,axis=1)
    to_be_removed = np.where(num_wrong > n * 0.3)   
    c = 0
    for i in to_be_removed[0]:        
        del samples[i -c]
        c = c+1
    new_samples = []
    if (len(samples)>=min_samples):
        new_samples = random.sample(samples,min_samples)
    return new_samples

def get_cross_val_lists(path):
    
    folders = os.listdir(path)
    
    subject_list= []
    for folder in folders:     
        subject_list.append(folder[:10].lower())

    unique_subject = list(set(subject_list))

    folds_dict={}
    for k_fold in [2,5,10]:
        n = len(unique_subject)/k_fold
        folds_dict['%s-fold' % k_fold] = {}
        for i in range(0,k_fold,1):
            folds_dict['%s-fold' % k_fold]['fold_%s' % i]= unique_subject[int(i*n):int((i+1)*n)]
    
    return folds_dict

def preprocess_data(folder,data_path):
    gyr_file = data_path + folder + '/Gyroscope.csv'
    
    if 'IMP' in folder.upper(): return 0,0
    
    if os.path.exists(gyr_file):  
        #print(folder)
        data_gyr = pd.read_csv(gyr_file)
        data_gyr.dropna()
        gyr = data_gyr.iloc[1:,1:4].values.astype(float)
        #print(gyr_file)
        #print(data_gyr)
        fr = get_frequency(data_gyr)
    
        #filter noise
        gyr_filt = butterfilt_data(gyr,fs = fr)
        #gyr_filt = sgfilt_data(gyr,win_len = np.floor(fr/2))

        #extract motion sequence
        gyr_cut, gyr_cut_fil = cut_data(gyr_filt,fr)
        #rotate
        if(len(gyr_cut)==0): return 0,0
        
        rot_data = rotate_data(gyr_cut)
        #rot_data = pca_man(gyr_cut)

        #find samples
        #this return a list with all samples and their data
        samples = sample_data(rot_data,fr)
        if (not samples): return 0,0
        
        #rescale samples to same frequency
        n = 400
        
        samples = resample_data(samples,n)

        #remove bad samples and data with too few samples
        min_samples = 3
        samples = remove_bad_samples(samples,n, min_samples)
        if (not samples): return 0,0
              
            
        if 'DO' in folder.upper():
            labels = np.full(min_samples,0)
        elif 'NO' in folder.upper():
            labels = np.full(min_samples,1)
        elif 'UP' in folder.upper():
            labels = np.full(min_samples,2)
        return samples, labels
    else: 
        return 0,0

def generate_numpy_arrays(fold_dict,data_path):
    folders = os.listdir(data_path)
    if not os.path.exists("arrays/"):
        os.makedirs("arrays/")
    else: 
        for k_fold in [2,5,10]:

            #accuracy_results = []
            for i in range(k_fold):
                training_subjects= []
                testing_subjects = []

                testing_subjects = fold_dict['%s-fold' % k_fold]['fold_%s'% i]
                print(i)
                for x in range(k_fold):
                    if x != i:
                        training_subjects += fold_dict['%s-fold' % k_fold]['fold_%s'% x]
                        print(x)

                train_data = []
                train_labels = []
                test_data = []
                test_labels = []

                for folder in folders:
                    if folder[:10].lower() in training_subjects:
                        samples, labels = preprocess_data(folder,data_path) 
                        if samples == 0: continue 
                        train_data.append(samples)
                        train_labels.append(labels)
                    elif folder[:10].lower() in testing_subjects:
                        samples, labels = preprocess_data(folder,data_path)
                        if samples == 0: continue
                        test_data.append(samples)
                        test_labels.append(labels)
                    else: continue

                training_data = np.concatenate((np.asarray(train_data)),axis=0)
                np.save('arrays/training_data_'+ '%s-fold_' % k_fold+'%s' %i,training_data)
                training_labels = np.concatenate(train_labels)
                np.save('arrays/training_labels_'+ '%s-fold_' % k_fold+'%s' %i,training_labels)
                testing_data = np.concatenate(test_data)
                np.save('arrays/testing_data_'+ '%s-fold_' % k_fold+'%s' %i,testing_data)
                testing_labels = np.concatenate(test_labels)
                np.save('arrays/testing_labels_'+ '%s-fold_' % k_fold+'%s' %i,testing_labels)
                print('Generated and saved arrays for %s-fold cross validation:' % k_fold + 'fold number %s' % i)           

def cross_validation_training(k_fold):
    
    accuracy_results = []
    for i in range(k_fold):
        training_data = np.load('arrays/training_data_'+ '%s-fold_' % k_fold+'%s' %i+'.npy')
        training_labels = np.load('arrays/training_labels_'+ '%s-fold_' % k_fold+'%s' %i+'.npy')
        testing_data = np.load('arrays/testing_data_'+ '%s-fold_' % k_fold+'%s' %i+'.npy')          
        testing_labels = np.load('arrays/testing_labels_'+ '%s-fold_' % k_fold+'%s' %i+'.npy')
        print('Training labels distribution:',np.unique(training_labels, return_index=False, return_inverse=False, return_counts=True, axis=None))
        print('Testing labels distribution:',np.unique(testing_labels, return_index=False, return_inverse=False, return_counts=True, axis=None))

        model = tf.keras.models.Sequential([
                tf.keras.layers.Flatten(input_shape=(400,3)),
                tf.keras.layers.Dense(100),
                tf.keras.layers.LeakyReLU(alpha=0.3),
                tf.keras.layers.Dropout(rate=0.25),
                tf.keras.layers.Dense(50),
                tf.keras.layers.LeakyReLU(alpha=0.3),
                tf.keras.layers.Dense(3, activation ='softmax')
                ])
        
        model_cnn = tf.keras.models.Sequential([tf.keras.layers.Conv1D(filters=16, kernel_size=3, activation='relu', 
                                                                       input_shape=(400,3)),
                                                #tf.keras.layers.BatchNormalization(),
                                                tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'),
                                                tf.keras.layers.Dropout(rate=0.10),
                                                tf.keras.layers.Flatten(),
                                                tf.keras.layers.Dense(50, activation ='relu'),
                                                tf.keras.layers.Dense(3, activation ='softmax')])
                        
        
        #Define early stop
        early_stop = tf.keras.callbacks.EarlyStopping(
                  monitor="val_accuracy",min_delta=0.05,
                  patience=15,
                  verbose=0)

        epochs = 70
        learning_rate = 5e-3
        decay_rate = learning_rate / epochs
        momentum = 0.85
        
        # Compile model
        #model.compile(loss='sparse_categorical_crossentropy', 
        #              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
        #              metrics=['accuracy'])
        
        model_cnn.compile(loss='sparse_categorical_crossentropy', 
                      optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate,momentum=momentum, decay=decay_rate, nesterov=False), 
                      metrics=['accuracy'])
        

        history = model_cnn.fit(training_data,training_labels,validation_data=(testing_data,testing_labels), 
                                batch_size = 8,epochs = epochs,
                               callbacks=early_stop)

        accuracy_results.append(np.max(history.history['val_accuracy']))
        #plt.plot(history.history['loss'])
        #plt.plot(history.history['val_loss'])
        #plt.title('Loss')
        #plt.ylabel('loss')
        #plt.xlabel('epoch')
        #plt.legend(['train', 'test'], loc='upper right')
        #plt.show()

    print('Best validation accuracy:', np.max(accuracy_results))
    print(accuracy_results)
    print('Mean validation accuracy:', np.mean(accuracy_results))
    return accuracy_results

### You only need to run the following cell once to generate and save the numpy arrays for training

In [112]:
data_path = 'C:/Users/admin/Documents/01_RWTH Courses/03_ WiSe21/CIE/Project A/Data/All data/Smartphone1Copy/'
#obtain distribution of subjects per fold
fold_dict = get_cross_val_lists(data_path)
#function that generates and saves the numpy arrays
generate_numpy_arrays(fold_dict,data_path)
print("Done!")

0
1
Generated and saved arrays for 2-fold cross validation:fold number 0
1
0
Generated and saved arrays for 2-fold cross validation:fold number 1
0
1
2
3
4
Generated and saved arrays for 5-fold cross validation:fold number 0
1
0
2
3
4
Generated and saved arrays for 5-fold cross validation:fold number 1
2
0
1
3
4
Generated and saved arrays for 5-fold cross validation:fold number 2
3
0
1
2
4
Generated and saved arrays for 5-fold cross validation:fold number 3
4
0
1
2
3
Generated and saved arrays for 5-fold cross validation:fold number 4
0
1
2
3
4
5
6
7
8
9
Generated and saved arrays for 10-fold cross validation:fold number 0
1
0
2
3
4
5
6
7
8
9
Generated and saved arrays for 10-fold cross validation:fold number 1
2
0
1
3
4
5
6
7
8
9
Generated and saved arrays for 10-fold cross validation:fold number 2
3
0
1
2
4
5
6
7
8
9
Generated and saved arrays for 10-fold cross validation:fold number 3
4
0
1
2
3
5
6
7
8
9
Generated and saved arrays for 10-fold cross validation:fold number 4
5
0
1
2
3

### Training

In [113]:
#In order to change the parameters you need to go to the cross_validation_training function
#There you can and should change the following parameters:
#number of neurons in each layer, number of layers, activation functions, patience on the early stopping,
#learning_rate, batch_size, epochs
#The relevant value for the accuracy of a model(or whatever metrics you use) is the mean value between all folds.

In [114]:
#Do one at a time if you prefer
accuracy_results_2fold = cross_validation_training(k_fold=2)
print('Mean accuracy for 2-fold: %s' % np.mean(accuracy_results_2fold))
print("-------------------------DONE------------------------")

Training labels distribution: (array([0, 1, 2]), array([144, 249, 186], dtype=int64))
Testing labels distribution: (array([0, 1, 2]), array([141, 255, 210], dtype=int64))
Epoch 1/70
73/73 [==============================] - 2s 17ms/step - loss: 0.9829 - accuracy: 0.4767 - val_loss: 0.7065 - val_accuracy: 0.6122
Epoch 2/70
73/73 [==============================] - 1s 14ms/step - loss: 0.7150 - accuracy: 0.6563 - val_loss: 0.5369 - val_accuracy: 0.8152
Epoch 3/70
73/73 [==============================] - 1s 19ms/step - loss: 0.6213 - accuracy: 0.7358 - val_loss: 0.5132 - val_accuracy: 0.7640
Epoch 4/70
73/73 [==============================] - 1s 20ms/step - loss: 0.5662 - accuracy: 0.7703 - val_loss: 0.5010 - val_accuracy: 0.7855
Epoch 5/70
73/73 [==============================] - 1s 18ms/step - loss: 0.5100 - accuracy: 0.7910 - val_loss: 0.4488 - val_accuracy: 0.8218
Epoch 6/70
73/73 [==============================] - 1s 13ms/step - loss: 0.4491 - accuracy: 0.8290 - val_loss: 0.4097 - val_

In [115]:
accuracy_results_5fold = cross_validation_training(k_fold=5)
print('Mean accuracy for 5-fold: %s' % np.mean(accuracy_results_5fold))
print("-------------------------DONE------------------------")

Training labels distribution: (array([0, 1, 2]), array([225, 414, 315], dtype=int64))
Testing labels distribution: (array([0, 1, 2]), array([60, 90, 81], dtype=int64))
Epoch 1/70
120/120 [==============================] - 3s 14ms/step - loss: 0.8668 - accuracy: 0.5660 - val_loss: 0.5258 - val_accuracy: 0.8052
Epoch 2/70
120/120 [==============================] - 1s 11ms/step - loss: 0.6012 - accuracy: 0.7579 - val_loss: 0.4912 - val_accuracy: 0.8095
Epoch 3/70
120/120 [==============================] - 1s 11ms/step - loss: 0.5397 - accuracy: 0.7820 - val_loss: 0.4191 - val_accuracy: 0.8528
Epoch 4/70
120/120 [==============================] - 1s 11ms/step - loss: 0.4938 - accuracy: 0.7935 - val_loss: 0.4458 - val_accuracy: 0.8442
Epoch 5/70
120/120 [==============================] - 1s 11ms/step - loss: 0.4381 - accuracy: 0.8333 - val_loss: 0.4534 - val_accuracy: 0.8182
Epoch 6/70
120/120 [==============================] - 1s 12ms/step - loss: 0.4081 - accuracy: 0.8291 - val_loss: 0.47

118/118 [==============================] - 1s 11ms/step - loss: 0.3880 - accuracy: 0.8403 - val_loss: 0.5746 - val_accuracy: 0.7561
Epoch 6/70
118/118 [==============================] - 1s 11ms/step - loss: 0.3582 - accuracy: 0.8562 - val_loss: 0.5223 - val_accuracy: 0.7967
Epoch 7/70
118/118 [==============================] - 1s 11ms/step - loss: 0.3351 - accuracy: 0.8637 - val_loss: 0.5284 - val_accuracy: 0.8252
Epoch 8/70
118/118 [==============================] - 1s 11ms/step - loss: 0.3111 - accuracy: 0.8786 - val_loss: 0.5746 - val_accuracy: 0.7886
Epoch 9/70
118/118 [==============================] - 1s 11ms/step - loss: 0.2719 - accuracy: 0.9020 - val_loss: 0.5424 - val_accuracy: 0.8130
Epoch 10/70
118/118 [==============================] - 1s 12ms/step - loss: 0.2311 - accuracy: 0.9063 - val_loss: 0.6265 - val_accuracy: 0.8049
Epoch 11/70
118/118 [==============================] - 1s 11ms/step - loss: 0.2066 - accuracy: 0.9212 - val_loss: 0.5924 - val_accuracy: 0.8171
Epoch 12

Epoch 7/70
121/121 [==============================] - 1s 10ms/step - loss: 0.3194 - accuracy: 0.8889 - val_loss: 0.4343 - val_accuracy: 0.8288
Epoch 8/70
121/121 [==============================] - 1s 10ms/step - loss: 0.3004 - accuracy: 0.8827 - val_loss: 0.5458 - val_accuracy: 0.7477
Epoch 9/70
121/121 [==============================] - 1s 11ms/step - loss: 0.2789 - accuracy: 0.8941 - val_loss: 0.4762 - val_accuracy: 0.8288
Epoch 10/70
121/121 [==============================] - 1s 10ms/step - loss: 0.2480 - accuracy: 0.9003 - val_loss: 0.4553 - val_accuracy: 0.8423
Epoch 11/70
121/121 [==============================] - 1s 11ms/step - loss: 0.2141 - accuracy: 0.9263 - val_loss: 0.4436 - val_accuracy: 0.8333
Epoch 12/70
121/121 [==============================] - 1s 10ms/step - loss: 0.2061 - accuracy: 0.9117 - val_loss: 0.5024 - val_accuracy: 0.8468
Epoch 13/70
121/121 [==============================] - 1s 11ms/step - loss: 0.1669 - accuracy: 0.9429 - val_loss: 0.4442 - val_accuracy: 0.

In [116]:
accuracy_results_10fold = cross_validation_training(k_fold=10)
print('Mean accuracy for 10-fold: %s' % np.mean(accuracy_results_10fold))
print("-------------------------DONE------------------------")

Training labels distribution: (array([0, 1, 2]), array([264, 462, 360], dtype=int64))
Testing labels distribution: (array([0, 1, 2]), array([21, 42, 36], dtype=int64))
Epoch 1/70
136/136 [==============================] - 3s 15ms/step - loss: 0.8142 - accuracy: 0.6077 - val_loss: 0.5453 - val_accuracy: 0.8081
Epoch 2/70
136/136 [==============================] - 2s 11ms/step - loss: 0.5690 - accuracy: 0.7716 - val_loss: 0.5636 - val_accuracy: 0.8283
Epoch 3/70
136/136 [==============================] - 1s 11ms/step - loss: 0.5056 - accuracy: 0.7947 - val_loss: 0.4254 - val_accuracy: 0.8283
Epoch 4/70
136/136 [==============================] - 1s 10ms/step - loss: 0.4506 - accuracy: 0.7993 - val_loss: 0.4438 - val_accuracy: 0.8687
Epoch 5/70
136/136 [==============================] - 1s 10ms/step - loss: 0.4232 - accuracy: 0.8278 - val_loss: 0.4436 - val_accuracy: 0.8081
Epoch 6/70
136/136 [==============================] - 1s 11ms/step - loss: 0.3735 - accuracy: 0.8481 - val_loss: 0.39

136/136 [==============================] - 1s 10ms/step - loss: 0.2303 - accuracy: 0.9125 - val_loss: 0.2898 - val_accuracy: 0.8586
Epoch 13/70
136/136 [==============================] - 1s 11ms/step - loss: 0.1887 - accuracy: 0.9263 - val_loss: 0.3148 - val_accuracy: 0.8788
Epoch 14/70
136/136 [==============================] - 1s 11ms/step - loss: 0.1210 - accuracy: 0.9586 - val_loss: 0.1452 - val_accuracy: 0.9293
Epoch 15/70
136/136 [==============================] - 2s 11ms/step - loss: 0.1375 - accuracy: 0.9457 - val_loss: 0.1979 - val_accuracy: 0.8687
Epoch 16/70
136/136 [==============================] - 1s 11ms/step - loss: 0.1146 - accuracy: 0.9567 - val_loss: 0.3134 - val_accuracy: 0.8889
Epoch 17/70
136/136 [==============================] - 1s 11ms/step - loss: 0.0933 - accuracy: 0.9632 - val_loss: 0.2305 - val_accuracy: 0.9192
Epoch 18/70
136/136 [==============================] - 1s 11ms/step - loss: 0.0711 - accuracy: 0.9751 - val_loss: 0.2049 - val_accuracy: 0.9293
Trai

134/134 [==============================] - 1s 11ms/step - loss: 0.5155 - accuracy: 0.7893 - val_loss: 0.7615 - val_accuracy: 0.7179
Epoch 3/70
134/134 [==============================] - 1s 11ms/step - loss: 0.4544 - accuracy: 0.8230 - val_loss: 0.7302 - val_accuracy: 0.7607
Epoch 4/70
134/134 [==============================] - 1s 11ms/step - loss: 0.4185 - accuracy: 0.8390 - val_loss: 0.8167 - val_accuracy: 0.6752
Epoch 5/70
134/134 [==============================] - 2s 11ms/step - loss: 0.3846 - accuracy: 0.8380 - val_loss: 0.6798 - val_accuracy: 0.7265
Epoch 6/70
134/134 [==============================] - 1s 11ms/step - loss: 0.3674 - accuracy: 0.8586 - val_loss: 0.7636 - val_accuracy: 0.7094
Epoch 7/70
134/134 [==============================] - 1s 11ms/step - loss: 0.3473 - accuracy: 0.8586 - val_loss: 0.7040 - val_accuracy: 0.7436
Epoch 8/70
134/134 [==============================] - 1s 11ms/step - loss: 0.2866 - accuracy: 0.8904 - val_loss: 0.7944 - val_accuracy: 0.6667
Epoch 9/70

Epoch 20/70
134/134 [==============================] - 1s 10ms/step - loss: 0.0897 - accuracy: 0.9757 - val_loss: 0.7485 - val_accuracy: 0.7778
Epoch 21/70
134/134 [==============================] - 1s 10ms/step - loss: 0.0855 - accuracy: 0.9794 - val_loss: 0.6060 - val_accuracy: 0.8376
Epoch 22/70
134/134 [==============================] - 1s 10ms/step - loss: 0.0963 - accuracy: 0.9691 - val_loss: 0.5308 - val_accuracy: 0.8376
Epoch 23/70
134/134 [==============================] - 1s 10ms/step - loss: 0.0562 - accuracy: 0.9822 - val_loss: 0.4319 - val_accuracy: 0.8803
Epoch 24/70
134/134 [==============================] - 1s 11ms/step - loss: 0.0875 - accuracy: 0.9719 - val_loss: 0.5142 - val_accuracy: 0.8291
Epoch 25/70
134/134 [==============================] - 1s 11ms/step - loss: 0.0658 - accuracy: 0.9860 - val_loss: 0.5171 - val_accuracy: 0.8632
Epoch 26/70
134/134 [==============================] - 1s 11ms/step - loss: 0.0499 - accuracy: 0.9850 - val_loss: 0.7396 - val_accuracy:

Epoch 15/70
137/137 [==============================] - 2s 11ms/step - loss: 0.0806 - accuracy: 0.9744 - val_loss: 0.3812 - val_accuracy: 0.8817
Epoch 16/70
137/137 [==============================] - 1s 11ms/step - loss: 0.0734 - accuracy: 0.9789 - val_loss: 0.4274 - val_accuracy: 0.8602
Epoch 17/70
137/137 [==============================] - 1s 11ms/step - loss: 0.0830 - accuracy: 0.9771 - val_loss: 0.4633 - val_accuracy: 0.8710
Epoch 18/70
137/137 [==============================] - 1s 11ms/step - loss: 0.0448 - accuracy: 0.9863 - val_loss: 0.5579 - val_accuracy: 0.7849
Best validation accuracy: 0.9292929172515869
[0.9090909361839294, 0.8863636255264282, 0.9292929172515869, 0.9251700639724731, 0.9147287011146545, 0.7606837749481201, 0.8211382031440735, 0.8888888955116272, 0.8914728760719299, 0.8924731016159058]
Mean validation accuracy: 0.8819303095340729
Mean accuracy for 10-fold: 0.8819303095340729
-------------------------DONE------------------------
